# Exploring Kalman Filters with the Inventory Simulator

This notebook provides an interactive introduction to Kalman filters through hands-on experimentation with the inventory simulator.

## What You'll Learn

- How Kalman filters estimate system state from partial observations
- The role of uncertainty in adaptive estimation
- How to tune Kalman filter parameters
- The trade-off between prediction and measurement
- How partial observability affects estimation

## Setup

First, let's import the necessary libraries and check that everything is installed correctly.

In [ ]:
# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Inventory simulator imports
from inventory_simulator import SimulatorConfig, SimulationRunner
from inventory_simulator.simulator import Simulator
from inventory_simulator.observer import Observer
from inventory_simulator.visualization import (
    plot_total_estimation_over_time,
    plot_kalman_uncertainty_over_time,
    plot_error_over_time,
    plot_shelf_comparison,
    plot_uncertainty_heatmap
)

# Configure matplotlib for notebook
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All imports successful!")
print("\nReady to explore Kalman filters!")

## Part 1: Your First Kalman Filter

Let's start with a simple simulation to see the Kalman filter in action.

### The Problem

You manage a warehouse with:
- **20 shelves** arranged in a circle
- **100 items** total (never changes - conservation law)
- Items randomly move between adjacent shelves
- You can only observe **one shelf at a time** (shelves 1-19 in round-robin)
- **Shelf #0 is never observed** (fundamental uncertainty)

**Goal**: Estimate the total number of items using a Kalman filter.

In [ ]:
# Create a basic configuration
config = SimulatorConfig(
    num_shelves=20,
    shelf_capacity=50,
    total_items=100,
    unobserved_shelf_id=0,
    movement_probability=0.01  # 1% of items move per timestep
)

print("Configuration:")
print(f"  Shelves: {config.num_shelves}")
print(f"  Total Items: {config.total_items}")
print(f"  Unobserved Shelf: #{config.unobserved_shelf_id}")
print(f"  Movement Probability: {config.movement_probability:.1%} per item per step")

In [ ]:
# Run the simulation
runner = SimulationRunner(config, seed=42)
results = runner.run(num_steps=500, report_interval=50)

print(f"\n✓ Simulation complete!")
print(f"  Steps run: 500")
print(f"  Analytics collected: {len(results.analytics_history)} reports")
print(f"  Events logged: {len(results.events_log)} events")

In [ ]:
# Display convergence results
print("\nKalman Filter Convergence:")
print("=" * 70)
print(f"{'Step':>6} {'Estimated':>10} {'Error':>8} {'Error %':>8} {'Uncertainty':>12}")
print("=" * 70)

for analytics in results.analytics_history:
    print(f"{analytics['step']:6d} "
          f"{analytics['estimated_total']:10.2f} "
          f"{analytics['total_error']:8.2f} "
          f"{analytics['total_error_pct']:8.2f}% "
          f"{analytics['kalman_uncertainty']:12.2f}")

final = results.analytics_history[-1]
print("=" * 70)
print(f"\nFinal Result: Estimated {final['estimated_total']:.2f} items (true: {config.total_items})")
print(f"Final Error: {final['total_error_pct']:.1f}%")

### Visualize the Convergence

Let's see how the Kalman filter learns over time.

In [ ]:
# Plot convergence
plot_total_estimation_over_time(
    results.analytics_history,
    true_total=config.total_items
)
plt.show()

In [ ]:
# Plot uncertainty reduction
plot_kalman_uncertainty_over_time(results.analytics_history)
plt.show()

In [ ]:
# Plot error over time
plot_error_over_time(results.analytics_history)
plt.show()

### Key Observations

Notice:
1. **Rapid Convergence**: The estimate goes from 0 to ~95-100 items within 100-200 steps
2. **Uncertainty Decreases**: The Kalman covariance (P) drops from 1000 to ~5-10
3. **Stable Error**: Error stabilizes around 4-6% despite continuous item movement
4. **Never Perfect**: We can't achieve 0% error because shelf #0 is never observed

---

## Part 2: Understanding the Kalman Filter Components

Let's peek inside the Kalman filter to see what's happening.

In [ ]:
# Create a fresh simulator and observer to track internals
simulator = Simulator(config, seed=42)
observer = Observer(config)

# Track Kalman filter variables
steps = []
kalman_states = []
kalman_covariances = []
kalman_gains = []
innovations = []
measurement_noises = []

for step in range(300):
    # Before update
    x_pred = observer._kf_state
    P_pred = observer._kf_covariance + observer._kf_process_noise
    
    # Simulate and observe
    simulator.step()
    observer.observe(simulator, step)
    
    # After update - calculate what happened
    z = observer._estimates['estimated_quantity'].sum()
    R = 10.0 + 0.5 * observer._estimates['uncertainty'].sum()
    K = P_pred / (P_pred + R)
    innovation = z - x_pred
    
    # Record
    if step % 10 == 0:
        steps.append(step)
        kalman_states.append(observer._kf_state)
        kalman_covariances.append(observer._kf_covariance)
        kalman_gains.append(K)
        innovations.append(innovation)
        measurement_noises.append(R)

print("✓ Collected Kalman filter internals for 300 steps")

### The Kalman Gain: Adaptive Weighting

The **Kalman gain** `K` determines how much to trust the new measurement vs. the prediction:
- `K = 1`: Trust measurement completely
- `K = 0`: Trust prediction completely
- `K` between 0 and 1: Balance both

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Kalman gain
ax1.plot(steps, kalman_gains, 'o-', color='darkred', linewidth=2, markersize=5)
ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='K=0.5 (Equal Trust)')
ax1.set_ylabel('Kalman Gain (K)', fontsize=12, fontweight='bold')
ax1.set_title('Kalman Gain Evolution: From Trust Measurements to Trust Predictions', 
              fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1])

# Uncertainties driving the Kalman gain
ax2.plot(steps, kalman_covariances, 's-', label='State Uncertainty (P)', linewidth=2, markersize=4)
ax2.plot(steps, measurement_noises, '^-', label='Measurement Noise (R)', linewidth=2, markersize=4)
ax2.set_xlabel('Step', fontsize=12, fontweight='bold')
ax2.set_ylabel('Uncertainty', fontsize=12, fontweight='bold')
ax2.set_title('K = P / (P + R): Uncertainty Components', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEarly (step 0):   K = {kalman_gains[0]:.3f} → Trust measurements heavily")
print(f"Late  (step 290): K = {kalman_gains[-1]:.3f} → Trust predictions more")

### Innovation: What We Learn from Each Measurement

The **innovation** `y = z - x_pred` is the "surprise" - how much the measurement differs from prediction.

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(steps, innovations, 'o-', color='purple', linewidth=2, markersize=4)
plt.axhline(y=0, color='black', linestyle='-', linewidth=1)
plt.fill_between(steps, 0, innovations, where=[i > 0 for i in innovations], 
                 color='green', alpha=0.3, label='Positive correction')
plt.fill_between(steps, innovations, 0, where=[i < 0 for i in innovations], 
                 color='red', alpha=0.3, label='Negative correction')
plt.xlabel('Step', fontsize=12, fontweight='bold')
plt.ylabel('Innovation (items)', fontsize=12, fontweight='bold')
plt.title('Innovation: Large Early (Learning), Small Late (Converged)', 
          fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Early innovations (steps 0-50):   Mean = {np.mean(innovations[:6]):.2f} items")
print(f"Late innovations  (steps 200-290): Mean = {np.mean(innovations[20:]):.2f} items")
print("\nLarge early innovations = rapid learning")
print("Small late innovations = converged, just tracking dynamics")

---

## Part 3: Experiments - Tune the Parameters!

Now it's your turn to experiment. Let's see how different parameters affect Kalman filter performance.

### Experiment 1: Movement Probability

How does system dynamics affect estimation?

In [ ]:
# Compare different movement probabilities
movement_probs = [0.001, 0.01, 0.05, 0.10]
results_by_movement = {}

for prob in movement_probs:
    config = SimulatorConfig(
        num_shelves=20,
        total_items=100,
        movement_probability=prob
    )
    runner = SimulationRunner(config, seed=42)
    results = runner.run(num_steps=500, report_interval=25)
    results_by_movement[prob] = results
    print(f"✓ Completed simulation with movement_probability = {prob:.1%}")

print("\nAll simulations complete!")

In [ ]:
# Plot comparison
plt.figure(figsize=(14, 6))

for prob, results in results_by_movement.items():
    steps = [a['step'] for a in results.analytics_history]
    errors = [a['total_error_pct'] for a in results.analytics_history]
    plt.plot(steps, errors, 'o-', label=f'{prob:.1%} movement', linewidth=2, markersize=4)

plt.axhline(y=10, color='red', linestyle='--', alpha=0.5, label='10% error threshold')
plt.xlabel('Step', fontsize=12, fontweight='bold')
plt.ylabel('Estimation Error (%)', fontsize=12, fontweight='bold')
plt.title('Effect of Movement Probability on Kalman Filter Performance', 
          fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nFinal Errors:")
for prob, results in results_by_movement.items():
    final_error = results.analytics_history[-1]['total_error_pct']
    print(f"  {prob:.1%} movement: {final_error:.1f}% error")

**Question**: What do you notice about the effect of movement probability?

*Hint*: Higher movement = more dynamics = harder to estimate accurately.

---

### Experiment 2: Number of Shelves

How does the observation cycle length affect convergence?

In [ ]:
# Compare different numbers of shelves
shelf_counts = [10, 20, 50, 100]
results_by_shelves = {}

for num_shelves in shelf_counts:
    config = SimulatorConfig(
        num_shelves=num_shelves,
        shelf_capacity=50,
        total_items=100,
        movement_probability=0.01
    )
    runner = SimulationRunner(config, seed=42)
    results = runner.run(num_steps=500, report_interval=25)
    results_by_shelves[num_shelves] = results
    print(f"✓ Completed simulation with {num_shelves} shelves")

print("\nAll simulations complete!")

In [ ]:
# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Error comparison
for num_shelves, results in results_by_shelves.items():
    steps = [a['step'] for a in results.analytics_history]
    errors = [a['total_error_pct'] for a in results.analytics_history]
    ax1.plot(steps, errors, 'o-', label=f'{num_shelves} shelves', linewidth=2, markersize=4)

ax1.set_xlabel('Step', fontsize=12, fontweight='bold')
ax1.set_ylabel('Estimation Error (%)', fontsize=12, fontweight='bold')
ax1.set_title('Convergence Speed vs. Number of Shelves', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Uncertainty comparison
for num_shelves, results in results_by_shelves.items():
    steps = [a['step'] for a in results.analytics_history]
    uncertainties = [a['kalman_uncertainty'] for a in results.analytics_history]
    ax2.plot(steps, uncertainties, 's-', label=f'{num_shelves} shelves', linewidth=2, markersize=4)

ax2.set_xlabel('Step', fontsize=12, fontweight='bold')
ax2.set_ylabel('Kalman Uncertainty (P)', fontsize=12, fontweight='bold')
ax2.set_title('Uncertainty Stabilization', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservation Cycle Lengths:")
for num_shelves in shelf_counts:
    cycle = num_shelves - 1  # Skip one shelf
    print(f"  {num_shelves} shelves → {cycle} steps to observe all (except shelf #0)")

**Question**: Why does increasing the number of shelves affect performance?

*Hint*: Longer round-robin cycle = more stale observations = higher measurement noise R.

---

### Experiment 3: Visualize Staleness

Let's see the partial observability in action.

In [ ]:
# Run a simulation and capture final state
config = SimulatorConfig(num_shelves=20, total_items=100)
runner = SimulationRunner(config, seed=42)
results = runner.run(num_steps=100, report_interval=100)

# Visualize shelf comparison
plot_shelf_comparison(
    results.final_ground_truth,
    results.final_estimates
)
plt.show()

# Visualize staleness
plot_uncertainty_heatmap(results.final_estimates)
plt.show()

In [ ]:
# Analyze the estimates
estimates_df = results.final_estimates.copy()
truth_df = results.final_ground_truth.copy()

# Merge for comparison
comparison = estimates_df.merge(truth_df, on='shelf_id', suffixes=('_est', '_true'))
comparison['error'] = comparison['estimated_quantity'] - comparison['quantity']
comparison['abs_error'] = comparison['error'].abs()

print("Shelf-by-Shelf Analysis (after 100 steps):")
print("=" * 80)
print(comparison[['shelf_id', 'quantity', 'estimated_quantity', 'error', 'uncertainty']].to_string(index=False))
print("=" * 80)

# Highlight shelf 0
shelf_0 = comparison[comparison['shelf_id'] == 0].iloc[0]
print(f"\nShelf #0 (NEVER OBSERVED):")
print(f"  True quantity: {shelf_0['quantity']} items")
print(f"  Estimated: {shelf_0['estimated_quantity']} items")
print(f"  Uncertainty: {shelf_0['uncertainty']} steps")
print(f"  → We've NEVER looked at this shelf, so estimate is 0 and uncertainty is maximal!")

# Observed shelves
observed = comparison[comparison['shelf_id'] != 0]
print(f"\nObserved Shelves (1-19):")
print(f"  Mean Absolute Error: {observed['abs_error'].mean():.2f} items")
print(f"  Max Absolute Error: {observed['abs_error'].max():.0f} items (shelf #{observed.loc[observed['abs_error'].idxmax(), 'shelf_id']})")
print(f"  Mean Uncertainty: {observed['uncertainty'].mean():.1f} steps")

---

## Part 4: Advanced Exploration

### Build Your Own Experiment

Try modifying the code below to explore different scenarios!

In [ ]:
# YOUR EXPERIMENT HERE
# Try changing:
# - num_shelves (how does 5 vs 100 shelves compare?)
# - total_items (does it scale to 1000 items?)
# - movement_probability (what if items rarely move? or move constantly?)
# - num_steps (how long does it take to converge?)

custom_config = SimulatorConfig(
    num_shelves=20,        # Try: 5, 10, 50, 100
    shelf_capacity=100,
    total_items=100,       # Try: 50, 200, 1000
    movement_probability=0.01  # Try: 0.001, 0.05, 0.20
)

runner = SimulationRunner(custom_config, seed=42)
custom_results = runner.run(num_steps=500, report_interval=50)

# Plot your results
plot_total_estimation_over_time(
    custom_results.analytics_history,
    true_total=custom_config.total_items
)
plt.show()

plot_error_over_time(custom_results.analytics_history)
plt.show()

final = custom_results.analytics_history[-1]
print(f"\nYour Experiment Results:")
print(f"  Final Error: {final['total_error_pct']:.1f}%")
print(f"  Final Uncertainty: {final['kalman_uncertainty']:.2f}")
print(f"  MAE (observed shelves): {final['mae']:.2f}")

### Challenge: Long-Term Behavior

What happens if we run for 10,000 steps?

In [ ]:
# Long simulation
config = SimulatorConfig(num_shelves=20, total_items=100, movement_probability=0.01)
runner = SimulationRunner(config, seed=42)
long_results = runner.run(num_steps=10000, report_interval=500)

print("Long-term behavior (10,000 steps):")
print("=" * 70)
for analytics in long_results.analytics_history:
    print(f"Step {analytics['step']:5d}: "
          f"Error = {analytics['total_error_pct']:5.2f}%, "
          f"Uncertainty = {analytics['kalman_uncertainty']:6.2f}")
print("=" * 70)

# Plot
plot_total_estimation_over_time(
    long_results.analytics_history,
    true_total=config.total_items
)
plt.show()

print("\nObservation: The filter remains stable over long periods!")
print("Error doesn't accumulate - the Kalman filter continuously adapts.")

---

## Part 5: Under the Hood - Direct Access

For advanced users: interact directly with Simulator and Observer.

In [ ]:
# Create simulator and observer manually
config = SimulatorConfig(num_shelves=10, total_items=50, movement_probability=0.02)
sim = Simulator(config, seed=123)
obs = Observer(config)

print("Initial State:")
print("\nSimulator (Ground Truth):")
print(sim.get_state())

print("\nObserver (Estimates - all zero initially):")
print(obs.get_estimates())
print(f"\nKalman Filter State: {obs.get_estimated_total():.2f} items")
print(f"Kalman Filter Uncertainty: {obs.get_total_uncertainty():.2f}")

In [ ]:
# Step-by-step simulation
print("Running 5 steps manually:\n")

for step in range(5):
    # Simulator moves items
    movement_event = sim.step()
    
    # Observer observes one shelf
    obs_event = obs.observe(sim, step)
    
    print(f"Step {step}:")
    print(f"  Observed shelf #{obs_event.observed_shelf}: "
          f"true={obs_event.true_quantity}, "
          f"was_estimated={obs_event.previous_estimate:.1f}")
    print(f"  Kalman estimate: {obs.get_estimated_total():.2f} items "
          f"(true: {config.total_items})")
    print(f"  Kalman uncertainty: {obs.get_total_uncertainty():.2f}")
    print()

In [ ]:
# Access Kalman filter internals
print("Kalman Filter Internals:")
print(f"  State (x): {obs._kf_state:.2f}")
print(f"  Covariance (P): {obs._kf_covariance:.2f}")
print(f"  Process Noise (Q): {obs._kf_process_noise}")
print(f"\nMeasurement Noise (R) = 10.0 + 0.5 × staleness")
print(f"  Current staleness: {obs._estimates['uncertainty'].sum():.1f}")
print(f"  Current R: {10.0 + 0.5 * obs._estimates['uncertainty'].sum():.2f}")

---

## Key Takeaways

### What You've Learned

1. **Kalman filters estimate hidden state from partial observations**
   - We can estimate total items without observing all shelves
   - Works despite never seeing shelf #0!

2. **Uncertainty drives adaptive behavior**
   - High uncertainty → trust measurements more (large K)
   - Low uncertainty → trust predictions more (small K)
   - Staleness increases measurement noise R

3. **Convergence requires time**
   - Filter needs ~100-200 observations to converge
   - Round-robin observation pattern creates lag
   - More shelves = longer convergence time

4. **Dynamics affect estimation**
   - Higher movement probability → harder to estimate
   - Filter balances tracking dynamics vs. noise

5. **Conservation constraints help**
   - Knowing total is constant (F=1) is powerful
   - Makes 1D state sufficient

### Next Steps

- Read the blog post (BLOG.md) for deeper theory
- Explore the source code in `src/inventory_simulator/`
- Try implementing your own estimator and compare to Kalman filter
- Extend to 2D state: estimate both total AND shelf #0 quantity

### Further Reading

- [How a Kalman filter works, in pictures](https://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/)
- [Kalman and Bayesian Filters in Python](https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python)
- [Kalman Filter Explained Simply](https://www.kalmanfilter.net/)

---

## Your Turn!

Use the cells below to experiment freely.

In [ ]:
# Free experimentation cell 1


In [ ]:
# Free experimentation cell 2


In [ ]:
# Free experimentation cell 3


---

**Happy filtering!** 🎯